# Imports

In [1]:
# import necessary python libraries and modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from scipy.stats import pearsonr
import datetime as dt
import seaborn as sns

# Load and Manipulate Data

##### Load Monthly Volume by Cohort Data

In [2]:
# Load data
df = pd.read_csv('Committed Volume by Cohort by Month.csv', parse_dates=['Month', 'Customer Cohort Month'])
df['Month'] = pd.to_datetime(df['Month'])
#df.set_index('Month', inplace=True)
df = df.sort_values(['Customer Cohort Month', 'Month'])
df

,Customer Cohort Month,Month,Volume
499,2018-01-31,2021-03-31,146000
602,2018-01-31,2021-04-30,154000
964,2018-01-31,2021-05-31,106000
51,2018-01-31,2021-06-30,850000
575,2018-01-31,2021-07-31,1162000
...,...,...,...
914,2023-02-28,2023-05-31,753000
169,2023-03-31,2023-03-31,78000
550,2023-03-31,2023-04-30,54000
712,2023-03-31,2023-05-31,123000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1005 entries, 499 to 226
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Customer Cohort Month  1005 non-null   datetime64[ns]
 1   Month                  1005 non-null   datetime64[ns]
 2   Volume                 1005 non-null   int64         
dtypes: datetime64[ns](2), int64(1)
memory usage: 31.4 KB


##### Create Variable for Percent Change in Volume and Remove NaNs (also removed % changes worse than -25% to prevent churn from increasing StDev)

In [4]:
# make pct change data
df['Volume % Change'] = df.groupby('Customer Cohort Month')['Volume'].pct_change()
df.dropna(inplace=True)
df = df[df['Volume % Change'] > -.25]
df

,Customer Cohort Month,Month,Volume,Volume % Change
602,2018-01-31,2021-04-30,154000,0.054795
51,2018-01-31,2021-06-30,850000,7.018868
575,2018-01-31,2021-07-31,1162000,0.367059
112,2018-01-31,2021-08-31,1393000,0.198795
996,2018-01-31,2021-09-30,1270000,-0.088299
...,...,...,...,...
164,2022-11-30,2023-05-31,376000,1.785185
308,2022-12-31,2023-04-30,73000,35.500000
714,2023-02-28,2023-03-31,560000,0.000000
914,2023-02-28,2023-05-31,753000,2.376682


##### Calculate Mean % Volume Change and Standard Deviation in Volume Change by Cohort

In [5]:
cohort_std_dev = df.groupby('Customer Cohort Month')['Volume % Change'].std()
cohort_std_dev

Customer Cohort Month
2018-01-31      1.746239
2018-03-31      0.217977
2018-06-30      0.337516
2018-07-31      0.411838
2018-08-31      0.531375
2018-09-30      0.642349
2018-10-31      0.575147
2018-11-30      0.158598
2018-12-31      0.233417
2019-01-31      4.960737
2019-02-28      2.400939
2019-03-31      0.773320
2019-04-30      0.845863
2019-05-31      0.253470
2019-07-31      0.188307
2019-08-31      0.310199
2019-09-30           NaN
2019-10-31      1.876683
2019-11-30      0.341242
2019-12-31      4.884737
2020-02-29      0.441916
2020-03-31      0.529220
2020-04-30      0.257590
2020-05-31           NaN
2020-06-30      0.524856
2020-07-31      0.866105
2020-09-30      0.361644
2020-10-31      0.456467
2020-11-30      0.314828
2020-12-31      0.263664
2021-01-31      0.688797
2021-02-28      0.761352
2021-03-31      4.660689
2021-04-30     77.900548
2021-05-31      0.838138
2021-06-30      1.635490
2021-07-31      0.648000
2021-08-31      0.954291
2021-09-30      0.707107
202

In [6]:
cohort_mean = df.groupby('Customer Cohort Month')['Volume % Change'].mean()
cohort_mean

Customer Cohort Month
2018-01-31     0.719324
2018-03-31    -0.029101
2018-06-30     0.126571
2018-07-31     0.148232
2018-08-31     0.305417
2018-09-30     0.188964
2018-10-31     0.233868
2018-11-30     0.068831
2018-12-31     0.079957
2019-01-31     3.506495
2019-02-28     1.984514
2019-03-31     0.291251
2019-04-30     0.597401
2019-05-31     0.088440
2019-07-31     0.099387
2019-08-31     0.123862
2019-09-30     5.000000
2019-10-31     0.502365
2019-11-30     0.100295
2019-12-31     3.002219
2020-02-29     0.209213
2020-03-31     0.082711
2020-04-30     0.044156
2020-05-31     0.000000
2020-06-30     0.137736
2020-07-31     0.209875
2020-09-30     0.265017
2020-10-31     0.173214
2020-11-30     0.068789
2020-12-31     0.080770
2021-01-31     0.343354
2021-02-28     0.544512
2021-03-31     1.135432
2021-04-30    19.894586
2021-05-31     0.384599
2021-06-30     0.595899
2021-07-31     0.350846
2021-08-31     0.427953
2021-09-30     0.500000
2021-10-31     7.064306
2021-11-30    36.8

#### Join to Create Mean and stDev by Cohort Data

In [7]:
df0 = pd.merge(cohort_std_dev, cohort_mean, left_on="Customer Cohort Month", right_on='Customer Cohort Month')
df0

,Volume % Change_x,Volume % Change_y
Customer Cohort Month,,
2018-01-31,1.746239,0.719324
2018-03-31,0.217977,-0.029101
2018-06-30,0.337516,0.126571
2018-07-31,0.411838,0.148232
2018-08-31,0.531375,0.305417
2018-09-30,0.642349,0.188964
2018-10-31,0.575147,0.233868
2018-11-30,0.158598,0.068831
2018-12-31,0.233417,0.079957


##### Load Customer Level volume Data and Create % Change in Volume Variable

In [8]:
df1 = pd.read_csv('Clean Committed Customer Data.csv', parse_dates=['Month', 'Customer Cohort Month'])
df1['Month'] = pd.to_datetime(df1['Month'])
df1 = df1[ ['CUSTOMER_ID-1', 'Month', 'Customer Cohort Month', 'Grand Total']].sort_values(['CUSTOMER_ID-1', 'Month', 'Customer Cohort Month'])
df1

,CUSTOMER_ID-1,Month,Customer Cohort Month,Grand Total
393,A00000003,2021-06-30,2018-01-31,740000
1265,A00000003,2021-07-31,2018-01-31,674000
1652,A00000003,2021-08-31,2018-01-31,847000
1924,A00000003,2021-09-30,2018-01-31,744000
1500,A00000003,2021-10-31,2018-01-31,721000
...,...,...,...,...
1977,A00002412,2023-05-31,2023-03-31,11000
1592,A00002429,2023-03-31,2023-03-31,2000
1819,A00002429,2023-04-30,2023-03-31,2000
428,A00002429,2023-05-31,2023-03-31,13000


In [9]:

df1['Volume % Change'] = df1.groupby('CUSTOMER_ID-1')['Grand Total'].pct_change()
df1.dropna(inplace=True)
df1

,CUSTOMER_ID-1,Month,Customer Cohort Month,Grand Total,Volume % Change
1265,A00000003,2021-07-31,2018-01-31,674000,-0.089189
1652,A00000003,2021-08-31,2018-01-31,847000,0.256677
1924,A00000003,2021-09-30,2018-01-31,744000,-0.121606
1500,A00000003,2021-10-31,2018-01-31,721000,-0.030914
1629,A00000003,2021-11-30,2018-01-31,761000,0.055479
...,...,...,...,...,...
1970,A00002404,2023-05-31,2023-03-31,35000,1.500000
603,A00002412,2023-04-30,2023-03-31,10000,-0.500000
1977,A00002412,2023-05-31,2023-03-31,11000,0.100000
1819,A00002429,2023-04-30,2023-03-31,2000,0.000000


##### Merge stDev and Customer Level Data

In [10]:
df2 = pd.merge(df1, df0, left_on='Customer Cohort Month', right_on='Customer Cohort Month')
df2

,CUSTOMER_ID-1,Month,Customer Cohort Month,Grand Total,Volume % Change,Volume % Change_x,Volume % Change_y
0,A00000003,2021-07-31,2018-01-31,674000,-0.089189,1.746239,0.719324
1,A00000003,2021-08-31,2018-01-31,847000,0.256677,1.746239,0.719324
2,A00000003,2021-09-30,2018-01-31,744000,-0.121606,1.746239,0.719324
3,A00000003,2021-10-31,2018-01-31,721000,-0.030914,1.746239,0.719324
4,A00000003,2021-11-30,2018-01-31,761000,0.055479,1.746239,0.719324
...,...,...,...,...,...,...,...
2050,A00002404,2023-05-31,2023-03-31,35000,1.500000,NaN,1.277778
2051,A00002412,2023-04-30,2023-03-31,10000,-0.500000,NaN,1.277778
2052,A00002412,2023-05-31,2023-03-31,11000,0.100000,NaN,1.277778
2053,A00002429,2023-04-30,2023-03-31,2000,0.000000,NaN,1.277778


##### Filter for Where Customer % Change in Volume is Within 1 stDev of Mean Volume Change for the Cohort

In [11]:
df3 = df2[((df2['Volume % Change_y']-df2['Volume % Change_x'])<=df2['Volume % Change']) & (df2['Volume % Change']<=(df2['Volume % Change_y']+df2['Volume % Change_x']))]
df4 = df3.sort_values(['CUSTOMER_ID-1', 'Month'])
df4

,CUSTOMER_ID-1,Month,Customer Cohort Month,Grand Total,Volume % Change,Volume % Change_x,Volume % Change_y
0,A00000003,2021-07-31,2018-01-31,674000,-0.089189,1.746239,0.719324
1,A00000003,2021-08-31,2018-01-31,847000,0.256677,1.746239,0.719324
2,A00000003,2021-09-30,2018-01-31,744000,-0.121606,1.746239,0.719324
3,A00000003,2021-10-31,2018-01-31,721000,-0.030914,1.746239,0.719324
4,A00000003,2021-11-30,2018-01-31,761000,0.055479,1.746239,0.719324
...,...,...,...,...,...,...,...
2041,A00001674,2023-03-31,2022-11-30,359000,0.014124,0.687069,0.701755
2042,A00001674,2023-04-30,2022-11-30,92000,-0.743733,0.687069,0.701755
2044,A00001719,2023-03-31,2023-02-28,560000,0.000000,1.680568,1.188341
2045,A00001719,2023-04-30,2023-02-28,223000,-0.601786,1.680568,1.188341


##### Find the First Month for Each Customer Where % Change in Volume Month-to-Month is Within Bounds (ramp month)

In [12]:
df5 = df4.groupby(['CUSTOMER_ID-1', 'Customer Cohort Month'])['Month'].min().reset_index()
df5['Ramp_Time_In_Months'] = (df5['Month'].dt.to_period('M') - df5['Customer Cohort Month'].dt.to_period('M')).apply(lambda x: (x.n))
df5

,CUSTOMER_ID-1,Customer Cohort Month,Month,Ramp_Time_In_Months
0,A00000003,2018-01-31,2021-07-31,42
1,A00000005,2018-01-31,2022-09-30,56
2,A00000006,2018-01-31,2021-08-31,43
3,A00000031,2018-01-31,2021-04-30,39
4,A00000039,2018-01-31,2021-12-31,47
...,...,...,...,...
134,A00001645,2022-10-31,2023-01-31,3
135,A00001648,2022-10-31,2022-12-31,2
136,A00001657,2022-11-30,2023-03-31,4
137,A00001674,2022-11-30,2022-12-31,1


In [13]:
df5.drop(columns=['Month'], inplace=True)

##### Reload Original Customer Data for Cohort Weighted Averages

In [14]:
# Reoad in intial data
df1 = pd.read_csv('Clean Committed Customer Data.csv', parse_dates=['Month', 'Customer Cohort Month'])
df1['Month'] = pd.to_datetime(df1['Month'])
df1 = df1 [ ['CUSTOMER_ID-1', 'Month', 'Customer Cohort Month', 'Grand Total']]
df1

,CUSTOMER_ID-1,Month,Customer Cohort Month,Grand Total
0,A00000340,2022-07-31,2020-06-30,1415000
1,A00000261,2022-11-30,2019-08-31,789000
2,A00000303,2022-06-30,2020-03-31,444000
3,A00001645,2023-03-31,2022-10-31,89000
4,A00000421,2021-06-30,2021-05-31,171000
...,...,...,...,...
2201,A00001456,2022-08-31,2021-11-30,7566000
2202,A00001477,2023-01-31,2022-01-31,6000
2203,A00001420,2022-07-31,2021-08-31,326000
2204,A00001416,2022-11-30,2021-06-30,2270000


In [15]:
df2 = df1.groupby(['CUSTOMER_ID-1', 'Customer Cohort Month'])['Grand Total'].sum()
df2

CUSTOMER_ID-1  Customer Cohort Month
A00000003      2018-01-31                37286000
A00000005      2018-01-31               101061000
A00000006      2018-01-31                 9985000
A00000031      2018-01-31                   87000
A00000039      2018-01-31                82161000
                                          ...    
A00002393      2023-03-31                  111000
A00002404      2023-03-31                   86000
A00002412      2023-03-31                   41000
A00002429      2023-03-31                   17000
A00002455      2023-05-31                   72000
Name: Grand Total, Length: 151, dtype: int64

##### Join Customer Volume Totals to Ramp Data and Calculate Weighted Average Ramp Time by Cohort

In [16]:
df3 = pd.merge(df5, df2, left_on=['CUSTOMER_ID-1', 'Customer Cohort Month'], right_on=['CUSTOMER_ID-1', 'Customer Cohort Month'])
df3

,CUSTOMER_ID-1,Customer Cohort Month,Ramp_Time_In_Months,Grand Total
0,A00000003,2018-01-31,42,37286000
1,A00000005,2018-01-31,56,101061000
2,A00000006,2018-01-31,43,9985000
3,A00000031,2018-01-31,39,87000
4,A00000039,2018-01-31,47,82161000
...,...,...,...,...
134,A00001645,2022-10-31,3,365000
135,A00001648,2022-10-31,2,3804000
136,A00001657,2022-11-30,4,516000
137,A00001674,2022-11-30,1,1745000


In [17]:
df4 = df3.groupby('Customer Cohort Month').apply(lambda x: np.average(x['Ramp_Time_In_Months'], weights=x['Grand Total']))
df4 = pd.DataFrame(df4)
df4['Weighted Average Ramp'] = df4[0]
df4.drop(columns=[0], inplace=True)
df4
#df4.to_csv('WeightedAverageRampByCohort.csv)

,Weighted Average Ramp
Customer Cohort Month,
2018-01-31,51.878036
2018-03-31,58.000000
2018-06-30,36.386486
2018-07-31,36.489613
2018-08-31,36.123785
2018-09-30,31.000000
2018-10-31,31.000000
2018-11-30,29.000000
2018-12-31,28.712548


##### Load Cohort Volume Totals Data and Join to Weighted Average Ramp Data

In [18]:
# Reload data
df = pd.read_csv('Committed Volume by Cohort by Month.csv', parse_dates=['Month', 'Customer Cohort Month'])
df['Month'] = pd.to_datetime(df['Month'])
#df.set_index('Month', inplace=True)
df = df.sort_values('Customer Cohort Month').groupby('Customer Cohort Month').sum()
df

,Volume
Customer Cohort Month,
2018-01-31,375979000
2018-03-31,41000
2018-06-30,103313000
2018-07-31,421578000
2018-08-31,10187000
2018-09-30,3882000
2018-10-31,52029000
2018-11-30,17357000
2018-12-31,122518000


##### Ignore More Recent Cohorts

In [19]:
df5 = pd.merge(df, df4, left_index=True, right_index=True)
df5

,Volume,Weighted Average Ramp
Customer Cohort Month,,
2018-01-31,375979000,51.878036
2018-03-31,41000,58.000000
2018-06-30,103313000,36.386486
2018-07-31,421578000,36.489613
2018-08-31,10187000,36.123785
2018-09-30,3882000,31.000000
2018-10-31,52029000,31.000000
2018-11-30,17357000,29.000000
2018-12-31,122518000,28.712548


##### Export to .csv file

In [20]:
#df5.to_csv('Weighted Average Ramp Time (Committed).csv', header=True)

##### Calculate Weighted Average of Weighted Averages (just a sense check; not as accurate as replicating the analysis at the cohort, not customer level, and weighting aggregate cohort ramp)

In [21]:
totalVol = df5['Volume'].sum()
df5['Weight'] = df5['Volume'] / totalVol
weightedAverageRamp = (df5['Weighted Average Ramp'] * df5['Weight']).sum()
weightedAverageRamp

14.93292861829684